In [1]:
clean_up = True
%run stdPackages.ipynb

In [2]:
import math, dash
import plotly.express as px
from plotly.subplots import make_subplots

# Simulations in partial equilibrium

In [3]:
ws = gams.GamsWorkspace(working_directory=d['work']) # specify where you want to run the GAMS models from (here the repository referred to in d['work'])
with open(f"{d['data']}\\A_GE","rb") as file:
    GE = pickle.load(file)
GE.write();

Define some standard plots for each shock:
1. Quantities/prices on durables (production module).
2. Investments (production module).
3. Total supply of domestically produced goods.
4. Total demand for all types of goods (summed over $s$).
5. Equilibrium prices.
6. Demand for all types of goods, for all sectors.

In [4]:
def pdSum(x, y):
    return x.groupby([i for i in x.index.names if i not in y]).sum() if pyDatabases.is_iterable(y) else x.groupby([i for i in x.index.names if i != y]).sum()
def adjustLabels(x, labels_, keepLoop):
    return adj.rc_pd(adjMultiIndex.applyMult(x, labels_).droplevel('l1'), keepLoop).unstack(keepLoop.name)
def stdPlots(m,GE,labels_,keepLoop, i = 'AIi', h = 'AHH'):
    mult_graphs()
    demand_n  = GE.get('nEqui').union(GE.get('nOut_f'))
    demand_sn = adj.rc_pd(m.out_db.get('qD'), demand_n).droplevel('t').index.unique()
    qd = adjustLabels(adj.rc_pd(m.out_db.get('sol_qD_shock'), ('and', [demand_sn, GE.get('txE')])), labels_, keepLoop)
    figs = {'qD_Durables': stdPlot(adjustLabels(adj.rc_pd(m.out_db.get('sol_qD_shock'), GE.get('dur_p')), labels_, keepLoop), GE.get('dur_p'), "$q$, durables"),
            'pD_Durables': stdPlot(adjustLabels(adj.rc_pd(m.out_db.get('sol_pD_shock'), GE.get('dur_p')), labels_, keepLoop), GE.get('dur_p'), "$p$, durables"),
            'qD_Investments': stdPlot(adjustLabels(adj.rc_pd(m.out_db.get('sol_qD_shock'), ('and', [GE.get('inv_p'), GE.get('txE')])), labels_, keepLoop), GE.get('inv_p'), "$q^d$, investments"),
            'qS': stdPlot(adjustLabels(adj.rc_pd(pdSum(m.out_db.get('sol_qS_shock'),'s'), ('and', [GE.get('nEqui'), GE.get('txE')])), labels_, keepLoop), GE.get('nEqui'), "$q^s$"),
            'qD': stdPlot(adjustLabels(adj.rc_pd(pdSum(m.out_db.get('sol_qD_shock'),'s'), ('and', [demand_n, GE.get('txE')])), labels_, keepLoop), demand_n, "$q^d"),
            'p': stdPlot(adjustLabels(adj.rc_pd(m.out_db.get('sol_p_shock'), GE.get('txE')), labels_, keepLoop), demand_n, "p"),
            'HH_savings': stdPlot(adjustLabels(pdSum(m.out_db.get('sol_vD_shock'),'n'), labels_, keepLoop), GE.get('s_h'), "$v$, households"),
            'HH_consump': stdPlot(adjustLabels(adj.rc_pd(m.out_db.get('sol_qD_shock'), ('and', [GE.get('output',m=h), GE.get('txE')])), labels_, keepLoop), GE.get('output',m=h), "$C$, households")}
    figs.update({f"qDs_{i}": stdPlot(adj.rc_pd(qd, GE.get(f"s_{i}")), adj.rc_pd(demand_sn, GE.get(f"s_{i}")), "q^d_s") for i in ('p','i','h','f','itory')})
    return figs
def stdPlot(df, c, title):
    nplots,nrows = len(c), math.ceil(len(c)/2)
    fig, axes = plt.subplots(nrows, min(nplots,2), figsize=(14,(4*nrows)));
    for i in range(nplots):
        ax = plt.subplot(nrows,min(nplots,2),i+1)
        df.xs(c[i], level = c.names if pyDatabases.is_iterable(c[i]) else c.name).plot(ax=ax, legend=False);
        ax.set_title(c[i])
    fig.legend(df.columns,ncol=3,loc=8, title = df.columns.name, frameon=True)
    # fig.suptitle(title,fontsize=15)
    fig.tight_layout()
    fig.subplots_adjust(bottom=0.35/nrows);
    return fig

### 1: An increase on the tax of $a$

*i. Set up target database:*

In [5]:
dbT = GpyDB(**{'name': 'TaxShock'})
dbT['tauS'] = adj.rc_pd(GE.get('tauS'), pd.Index(['a'],name='n')) + 0.25
# dbT['tauD'] = pd.Series(0.25, index = pd.MultiIndex.from_product([GE.get('t'), GE.get('s'), pd.Index(['a'], name = 'n')]), name = 'tauD')
extractSol = {k:None for k in ('qS','qD','pD','p')}
extractSol['vD'] = GE.g('svngs')
labels = pd.Index(np.linspace(0, 0.25, 26), name = """$\Delta \\tau^S_a$""")

*ii. Solve:*

In [6]:
m_tax, cp_tax = GE.sneakySolve(dbT, ws = ws, loop_kwargs = {'extractSol': extractSol, 'n': 26}, **{'cns': 'CONOPT4'})

*iii. Plot:*

In [7]:
%%capture
keepLoop = pd.Index([labels[0], labels[13], labels[25]], name = labels.name) # which labels in the loop are we plotting
labels_ = pd.MultiIndex.from_arrays([m_tax.out_db.get('l1'), labels]) 
figs_tax = stdPlots(m_tax, GE, labels_, keepLoop) # store figures

*Export:*

In [8]:
[f.savefig(f"{d['figs']}\\CGE_A\\GE_{dbT.name}_{k}.pdf", edgecolor='k') for k,f in figs_tax.items()];

### 2: A decrease of initial capital of 25%

*i. Set up target database:*

In [9]:
dbT = GpyDB(**{'name': 'CapitalShockLower'})
dbT['qD'] = adj.rc_pd(GE.get('qD'), ('and', [GE.get('dur_p'), GE.get('t0')])) * 0.75
labels = pd.Index(range(26), name = """% drop in $K_0$""")

*ii. Solve:*

In [10]:
m_kl, cp_kl = GE.sneakySolve(dbT, ws = ws, loop_kwargs = {'extractSol': extractSol, 'n': 26}, **{'cns': 'CONOPT4'})

*iii. Plot:*

In [11]:
%%capture
keepLoop = pd.Index([labels[0], labels[13], labels[25]], name = labels.name) # which labels in the loop are we plotting
labels_ = pd.MultiIndex.from_arrays([m_kl.out_db.get('l1'), labels]) 
figs_kl = stdPlots(m_kl, GE, labels_, keepLoop) # store figures

*Export:*

In [12]:
[f.savefig(f"{d['figs']}\\CGE_A\\GE_{dbT.name}_{k}.pdf", edgecolor='k') for k,f in figs_kl.items()];

### 3: An increase in initial capital of 25%

*i. Set up target database:*

In [13]:
dbT = GpyDB(**{'name': 'CapitalShockUpper'})
dbT['qD'] = adj.rc_pd(GE.get('qD'), ('and', [GE.get('dur_p'), GE.get('t0')])) * 1.25
labels = pd.Index(range(26), name = """% increase in $K_0$""")

*ii. Solve:*

In [14]:
m_ku, cp_ku = GE.sneakySolve(dbT, ws = ws, loop_kwargs = {'extractSol': extractSol, 'n': 26}, **{'cns': 'CONOPT4'})

*iii. Plot:*

In [15]:
%%capture
keepLoop = pd.Index([labels[0], labels[13], labels[25]], name = labels.name) # which labels in the loop are we plotting
labels_ = pd.MultiIndex.from_arrays([m_ku.out_db.get('l1'), labels]) 
figs_ku = stdPlots(m_ku, GE, labels_, keepLoop) # store figures

*Export:*

In [16]:
[f.savefig(f"{d['figs']}\\CGE_A\\GE_{dbT.name}_{k}.pdf", edgecolor='k') for k,f in figs_ku.items()];

### 4: A temporarily higher depreciation rate

*i. Set up target database:*

In [17]:
dbT = GpyDB(**{'name': 'TempDepr'})
dbT['rDepr'] = GE.get('rDepr').add(pd.Series(np.linspace(0.05, 0, 11), index = GE.get('t')[0:11]), fill_value=0)
labels = pd.Index(np.round(np.linspace(0, 0.05, 26),3), name = """$\Delta \delta_{t_0}$""")

*ii. Solve:*

In [18]:
m_tempDepr, cp_tempDepr = GE.sneakySolve(dbT, ws = ws, loop_kwargs = {'extractSol': extractSol, 'n': 26}, **{'cns': 'CONOPT4'})

*iii. Plot:*

In [19]:
%%capture
keepLoop = pd.Index([labels[0], labels[13], labels[25]], name = labels.name) # which labels in the loop are we plotting
labels_ = pd.MultiIndex.from_arrays([m_tempDepr.out_db.get('l1'), labels]) 
figs_tempDepr = stdPlots(m_tempDepr, GE, labels_, keepLoop) # store figures

*Export:*

In [20]:
[f.savefig(f"{d['figs']}\\CGE_A\\GE_{dbT.name}_{k}.pdf", edgecolor='k') for k,f in figs_tempDepr.items()];